In [1]:
import pandas as pd
import numpy as np
from backtesting import Backtest, Strategy
from data_storage import create_connection
from stockstats import StockDataFrame

In [2]:
connection = create_connection("../database/crypto_billionairs.db")

In [3]:
table = "BTCUSDT_1m_complete_history_long_short"

In [4]:
df_long = pd.read_sql_query(f'select * from {table}', connection)

In [5]:
df_backtesting = pd.DataFrame()
if "5m" not in table:
    #df_backtesting["time"] = df_long["time"]
    df_backtesting[['Open', 'High', 'Low', 'Close', "Volume"]] = df_long[['open', 'high', 'low', 'close', 'volume']]
    #df_backtesting = df_backtesting.set_index("time")

else:
    df_backtesting[['Open', 'High', 'Low', 'Close', "Volume"]] = df_long[['open', 'high', 'low', 'close', 'volume']]

In [6]:
df_backtesting = df_backtesting[1200000:]
df_long = df_long[1200000:]

In [7]:
def calculate_osc(data):
    
   
    df_temp = pd.DataFrame()    
    stock = StockDataFrame.retype(data)
    
    df_temp["stochastic_oscillator"] = stock.get('kdjk')
    
    return df_temp["stochastic_oscillator"]

def calucalte_rsi(data):
    
    df_temp = pd.DataFrame()    
    stock = StockDataFrame.retype(data)
    
    df_temp["relative_strength_index"] = stock.get("rsi_30")
    
    return df_temp["relative_strength_index"]

def momentum(df, lag):
    return df["close"].pct_change(periods=lag)

def init_close_long_signal(df):
    
    return df_long['close_indicator']

def init_buy_signal(df):
    
    return df_long['buy_indicator']

def init_short_signal(df):
    
    return df_long['short_indicator']

def init_close_short_signal(df):
    
    return df_long['short_close_indicator']
    

In [8]:
class OscRsi_5m_long_short(Strategy):
   
    lag = 4
    
    def init(self):
        # compute the rsi and stochastic oscillator with stockstats and return the buy signal of the current row
        
        self.osc = self.I(calculate_osc, self.data.df)
        self.rsi = self.I(calucalte_rsi, self.data.df)
        self.buy_init = self.I(init_buy_signal, self.data.df)
        #self.close_buy_init = self.I(init_close_long_signal, self.data.df)
        self.short_init = self.I(init_short_signal, self.data.df)
        #self.close_short_init = self.I(init_close_short_signal, self.data.df)
        self.data.df[['Open', 'High', 'Low', 'Close', "Volume",]] = self.data.df[['open', 'high', 'low', 'close', "volume"]]
        self.returns = self.I(momentum, self.data.df, self.lag)
        self.data.df.drop(self.data.df.columns.difference(['Open', 'High', 'Low', 'Close', "Volume", "kdjk", "rsi_30"]), 1, inplace=True)
        
       
    
    def next(self):
        
        if self.buy_init[-1] == 1 and self.position.is_long is False:
            self.position.close()
            self.buy()
            
        # elif self.close_buy_init[-1] == 1 and self.position.is_long:
        #     self.position.close()
             
             
        elif self.short_init[-1] == -1 and self.position.is_short is False:
            self.position.close()
            self.sell()
            
        # elif self.close_short_init[-1] == 1 and self.position.is_short:
        #      self.position.close()


In [9]:
bt = Backtest(df_backtesting, OscRsi_5m_long_short, cash=100_000, commission=.001)
stats = bt.run()
stats

<ipython-input-9-96704af4b3b0>:1: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(df_backtesting, OscRsi_5m_long_short, cash=100_000, commission=.001)


Start                                 1.2e+06
End                               3.97947e+06
Duration                          2.77947e+06
Exposure Time [%]                     99.1168
Equity Final [$]                       342938
Equity Peak [$]                        773898
Return [%]                            242.938
Buy & Hold Return [%]                 5867.11
Return (Ann.) [%]                           0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                                0
Max. Drawdown [%]                    -73.4746
Avg. Drawdown [%]                    -2.33718
Max. Drawdown Duration                 801354
Avg. Drawdown Duration                3974.77
# Trades                                  298
Win Rate [%]                          35.2349
Best Trade [%]                        123.693
Worst Trade [%]                      -17.4264
Avg. Trade [%]                    

In [10]:
# bt.plot()

In [11]:
# %%time 
# stats = bt.optimize(lag=[12, 24, 48, 96, 120, 144, 168],
#                     sell_ = np.arange(60, 90, 3).tolist(),#sell_threshold = np.arange(0, -0.15, -0.01).tolist(),
#                     rsi_lower_bound = np.arange(10, 40, 3).tolist(),
#                     rsi_upper_bound = np.arange(12, 50, 3).tolist(),
#                     #osc_lower_bound = np.arange(10, 90, 5).tolist(),
#                     #osc_upper_bound = np.arange(10, 100, 5).tolist(),
#                     #maximize='Equity Final [$]')#,
#                     constraint= lambda param: param.rsi_lower_bound < param.rsi_upper_bound)
# stats

In [12]:
stats._strategy

<Strategy OscRsi_5m_long_short>

In [13]:
stats['_trades']


,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-115,24548,24698,869.13000,998.99,-14933.90000,-0.149414,1224548,1224698,150
1,85,24698,25884,999.98899,887.76,-9539.46415,-0.112230,1224698,1225884,1186
2,-85,25884,43033,886.87224,895.25,-712.10960,-0.009446,1225884,1243033,17149
3,83,43033,114095,896.14525,1153.80,21385.34425,0.287514,1243033,1314095,71062
4,-83,114095,116316,1152.64620,1334.90,-15127.06540,-0.158118,1314095,1316316,2221
...,...,...,...,...,...,...,...,...,...,...
293,7,2687205,2715724,37988.91096,41821.00,26824.62328,0.100874,3887205,3915724,28519
294,-7,2715724,2729724,41779.17900,39369.32,16869.01300,0.057681,3915724,3929724,14000
295,8,2729724,2741948,39408.68932,38545.09,-6908.79456,-0.021914,3929724,3941948,12224
296,-8,2741948,2748506,38506.54491,40137.00,-13043.64072,-0.042342,3941948,3948506,6558


In [14]:
trades = pd.DataFrame(stats['_trades'])

In [15]:
trades.to_sql(f"trades_{table}", connection, if_exists="replace")

In [16]:
trades.to_excel(f"trades_{table}.xlsx")

In [17]:
from risk_metrics import Risk_Metrics

In [18]:
Risk_Metrics(trades, df_long, 0).sharpe_ratio("momentum_1day")

297    0.717987
Name: ReturnPct, dtype: float64

In [19]:
Risk_Metrics(trades, df_long, 0).sortino_ratio("momentum_1day")

297    7.331461
Name: ReturnPct, dtype: float64

In [20]:
Risk_Metrics(trades, df_long, 0).max_drawdown()

0.5590275813247902

In [25]:
Risk_Metrics(trades, df_long, 0).calmar_ratio(stats["Return [%]"]) 

82.17772851547416